# OmniEQ Python API
## Initialize

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from omnieq.client import Client as oeq
from omnieq.utils.pd_funcs import filter_future, filter_past, days_until
import pandas as pd

In [24]:
UNDERLYING_SYMBOL = input('Underlying Symbol: ')

In [25]:
oeq.set_token(input('OmniEQ Token: '))

## Chain Expirations
### Get all expirations by underlying symbol

In [26]:
expirations = oeq.chain_expirations(UNDERLYING_SYMBOL)
expirations[:5]

,expiry,series
0,2023-01-20,Monthly
1,2022-01-21,Monthly
2,2021-07-16,Monthly
3,2021-04-16,Monthly
4,2021-02-26,Weekly


### Helper functions for past, future, and days
#### Past and future filters

In [27]:
expirations_future = filter_future(expirations, 'expiry')
expirations_past = filter_past(expirations, 'expiry')

expirations_future['state'] = 'future'
expirations_past['state'] = 'past'

pd.concat([expirations_future[:2], expirations_past[:2]])

,expiry,series,state
0,2023-01-20,Monthly,future
1,2022-01-21,Monthly,future
11,2021-01-08,Weekly,past
12,2020-12-31,Weekly,past


#### Days until

In [28]:
df = expirations.copy()
df['days_to_expiration'] = days_until(df, 'expiry')
df[:5]

,expiry,series,days_to_expiration
0,2023-01-20,Monthly,-741.319209
1,2022-01-21,Monthly,-377.319209
2,2021-07-16,Monthly,-188.319209
3,2021-04-16,Monthly,-97.319209
4,2021-02-26,Weekly,-48.319209


## Chain EODs

For a single expiration ...

In [29]:
t_expiration = expirations.iloc[0]['expiry']
t_expiration

'2023-01-20'

In [30]:
eods = oeq.chain_eods(UNDERLYING_SYMBOL, t_expiration, 5)
eods[:5]

expiry  type  strike  lastPrice  bidPrice  \
symbol           eod_date                                                    
BHC20Jan23C10.00 2021-01-04  2023-01-20  CALL    10.0       12.7     11.95   
                 2021-01-05  2023-01-20  CALL    10.0       12.7     12.50   
                 2021-01-06  2023-01-20  CALL    10.0       14.0     13.05   
                 2021-01-07  2023-01-20  CALL    10.0       14.0     13.70   
                 2021-01-08  2023-01-20  CALL    10.0       14.0     13.80   

                             askPrice  bidSize  askSize  volume  openInterest  \
symbol           eod_date                                                       
BHC20Jan23C10.00 2021-01-04     13.15       32       31       5           557   
                 2021-01-05     13.70       26       26       0           562   
                 2021-01-06     14.25       98      155     100           562   
                 2021-01-07     14.45       22       29       7           484   
                 2021-01-08     15.20      180      158       0           482   

                             impliedVolatility     delta     theta     gamma  \
symbol           eod_date                                                      
BHC20Jan23C10.00 2021-01-04          57.032130  0.917396 -0.001676  0.009337   
                 2021-01-05          68.109520  0.917086 -0.001681  0.009374   
                 2021-01-06          69.493099  0.922514 -0.001630  0.008747   
                 2021-01-07          64.986591  0.929239 -0.001567  0.007942   
                 2021-01-08          62.140894  0.934602 -0.001509  0.007318   

                                 vega       rho  
symbol           eod_date                        
BHC20Jan23C10.00 2021-01-04  0.046790  0.150236  
                 2021-01-05  0.046825  0.149884  
                 2021-01-06  0.045558  0.152399  
                 2021-01-07  0.043898  0.155501  
                 2021-01-08  0.042414  0.158087

For multiple expirations ...

In [31]:
t_expirations = list(expirations.iloc[0:3]['expiry'].unique())
t_expirations

['2023-01-20', '2022-01-21', '2021-07-16']

In [32]:
eods = oeq.chain_eods(UNDERLYING_SYMBOL, t_expirations, 5)
eods.sample(n=10)

,,expiry,type,strike,lastPrice,bidPrice,askPrice,bidSize,askSize,volume,openInterest,impliedVolatility,delta,theta,gamma,vega,rho
symbol,eod_date,,,,,,,,,,,,,,,,
BHC20Jan23C5.00,2021-01-05,2023-01-20,CALL,5.0,16.30,16.70,17.95,25,25,0,2,94.879130,0.989219,-0.000319,0.001746,0.008724,0.095729
BHC21Jan22C35.00,2021-01-04,2022-01-21,CALL,35.0,0.86,0.50,1.37,724,616,2,1724,45.159543,0.266021,-0.005012,0.028098,0.072194,0.044499
BHC21Jan22C25.00,2021-01-06,2022-01-21,CALL,25.0,3.40,3.10,3.55,811,563,1783,5138,49.446715,0.512914,-0.006222,0.033608,0.089522,0.079770
BHC20Jan23P27.00,2021-01-07,2023-01-20,PUT,27.0,NaN,8.40,9.05,88,61,0,0,46.014978,-0.441599,-0.004479,0.023152,0.127962,-0.399464
BHC21Jan22P5.00,2021-01-06,2022-01-21,PUT,5.0,0.10,NaN,0.16,0,520,0,227,85.766719,-0.001292,-0.000066,0.000359,0.000956,-0.000349
BHC16Jul21P12.00,2021-01-08,2021-07-16,PUT,12.0,0.30,NaN,0.45,0,31,0,13,74.263237,-0.025438,-0.001372,0.006736,0.009984,-0.003596
BHC16Jul21C21.00,2021-01-08,2021-07-16,CALL,21.0,4.64,4.05,4.75,494,520,61,2270,48.697298,0.680527,-0.008303,0.040593,0.060166,0.058636
BHC16Jul21C27.00,2021-01-04,2021-07-16,CALL,27.0,1.21,0.83,1.12,624,6,10,23,42.167799,0.345675,-0.007910,0.044355,0.057726,0.031057
BHC16Jul21C22.00,2021-01-05,2021-07-16,CALL,22.0,2.88,2.72,2.88,463,10,88,1514,48.921116,0.550928,-0.008511,0.047744,0.061704,0.046286
